# Collisons

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [47]:
import numpy as np
from numpy.linalg import norm
# import pandas as pd
# import plotting modules
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
# from matplotlib.offsetbox import AnchoredText
%matplotlib inline

from galaxy.db import DB
from galaxy.galaxies import Galaxies
from galaxy.galaxy import Galaxy
from galaxy.timecourse import TimeCourse
from galaxy.plots import Plots

In [48]:
import mpl_scatter_density

# Make the norm object to define the image stretch
from astropy.visualization import LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
lognorm = ImageNormalize(vmin=0., vmax=1000, stretch=LogStretch())

In [6]:
#adjust tick label font size
label_size = 14
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

In [36]:
def get_xyz(snap, ptype=2):
    MW = Galaxy('MW', snap, usesql=True, ptype=2)
    M31 = Galaxy('M31', snap, usesql=True, ptype=2)
    M33 = Galaxy('M33', snap, usesql=True, ptype=2)

    MW_xyz = [MW.data[xi] for xi in ('x','y','z')]
    M31_xyz = [M31.data[xi] for xi in ('x','y','z')]
    M33_xyz = [M33.data[xi] for xi in ('x','y','z')]
    MW_xyz = np.array(MW_xyz)
    M31_xyz = np.array(M31_xyz)
    M33_xyz = np.array(M33_xyz)

    disks_raw = np.concatenate([MW_xyz, M31_xyz, M33_xyz], axis=1)
    centroid = np.mean(disks_raw, axis=1)
    
    t = MW.time.value / 1000
    return disks_raw - centroid[:,np.newaxis], t

In [68]:
def plot_density_views(disks, title, xlim=150, ylim=150, pngout=False, fname=None):
    fig = plt.figure()
    DPI = fig.get_dpi()
    fig.set_size_inches(1400.0/float(DPI),1400.0/float(DPI))
    fontsize = 16

    # top left
    ax0 = fig.add_subplot(2, 2, 1, projection='scatter_density')
    ax0.scatter_density(disks[0], disks[1], norm=lognorm)

    ax0.set_xlim(-xlim, xlim)
    ax0.set_ylim(-ylim, ylim)

    # Add axis labels (standard pyplot)
    ax0.set_xlabel('x (kpc)', fontsize=fontsize)
    ax0.set_ylabel('y (kpc)', fontsize=fontsize)

    # top right
    ax1 = fig.add_subplot(2, 2, 2, projection='scatter_density')
    ax1.scatter_density(disks[0], disks[2], norm=lognorm)

    ax1.set_xlim(-xlim, xlim)
    ax1.set_ylim(-ylim, ylim)

    # Add axis labels (standard pyplot)
    ax1.set_xlabel('x (kpc)', fontsize=fontsize)
    ax1.set_ylabel('z (kpc)', fontsize=fontsize)

    # bottom left
    ax2 = fig.add_subplot(2, 2, 3, projection='scatter_density')
    ax2.scatter_density(disks[1], disks[2], norm=lognorm)

    ax2.set_xlim(-xlim, xlim)
    ax2.set_ylim(-ylim, ylim)

    # Add axis labels (standard pyplot)
    ax2.set_xlabel('y (kpc)', fontsize=fontsize)
    ax2.set_ylabel('z (kpc)', fontsize=fontsize)

    # bottom right
    ax3 = fig.add_subplot(2, 2, 4)
    ax3.axis('off')
    ax3.text(0.1, 0.5, title, fontsize=28, transform=ax3.transAxes)
    plt.tight_layout()
    
    # Save file
    if pngout:
        plt.savefig(fname, dpi='figure')
        plt.close();  

In [51]:
# my non-standard class
tc = TimeCourse()

# the next 2 lines only work on my machine, as it needs the postgres database
com_MW = tc.read_com_db(galaxy='MW')
com_M33 = tc.read_com_db(galaxy='M33')
com_M31 = tc.read_com_db(galaxy='M31')

# need the time values, not the same length as the sim values
t_sim = com_M33['t']

# calculate separations
xyz_MW = np.array([com_MW[xi] for xi in ('x','y','z')])
xyz_M31 = np.array([com_M31[xi] for xi in ('x','y','z')])
xyz_M33 = np.array([com_M33[xi] for xi in ('x','y','z')])
r_MW_M31 = norm(xyz_MW - xyz_M31, axis=0)
r_M33_M31 = norm(xyz_M33 - xyz_M31, axis=0)
r_MW_M33 = norm(xyz_MW - xyz_M33, axis=0)

In [53]:
r_MW_M31.shape

(802,)

In [73]:
for snap in np.arange(0, 200):
    disks, t = get_xyz(snap)
    title = f"\n\nSnap: {snap:03}\nElapsed time: {t:5.3f} Gyr\n\nSeparations:\n"
    title += f"   MW-M31 : {r_MW_M31[snap]:7.1f} kpc\n"
    title += f"  M33-M31 : {r_M33_M31[snap]:7.1f} kpc\n"
    title += f"   MW-M33 : {r_MW_M33[snap]:7.1f} kpc"

    plot_density_views(disks, title, pngout=True, fname=f"collision_{snap:03}.png")
    print(snap, end=' ')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 

In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))